In [1]:

import spacy
print(spacy.__version__)
!python -m spacy download en_core_web_sm

3.7.4
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB 1.3 MB/s eta 0:00:10
     --------------------------------------- 0.0/12.8 MB 487.6 kB/s eta 0:00:27
     --------------------------------------- 0.1/12.8 MB 653.6 kB/s eta 0:00:20
     --------------------------------------- 0.1/12.8 MB 819.2 kB/s eta 0:00:16
      --------------------------------------- 0.2/12.8 MB 1.0 MB/s eta 0:00:13
      --------------------------------------- 0.2/12.8 MB 1.1 MB/s eta 0:00:12
      --------------------------------------- 0.3/12.8 MB 1.2 MB/s eta 0:00:11
     - -------------------------------------- 0.4/12.8 MB 1.3 MB/s eta 0:00:10
     - -------------------------------------- 0.4/12.8 MB 1.2 MB/s eta 0:00:11
     - -------------------------------------- 0.5/12.8 MB 1.3 MB/s eta 0:00:10
     - -------------------------------------- 0.6/12.8 MB 1.3 MB/s eta 0:00:10
     -- ------------------------------------- 0.6/

3) charger et explorer les données

In [2]:
import pandas as pd
import nltk
import spacy

# Télécharger les ressources nécessaires pour NLTK
nltk.download('punkt')
nltk.download('stopwords')

# Charger les stopwords de NLTK
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Vérifier et télécharger le modèle de spaCy s'il n'est pas installé
try:
    nlp = spacy.load('en_core_web_sm')
except OSError:
    print("Le modèle 'en_core_web_sm' n'est pas installé. Téléchargement en cours...")
    from spacy.cli import download
    download('en_core_web_sm')
    nlp = spacy.load('en_core_web_sm')

# Charger les données
file_path = 'train-v2.0.json'   
df = pd.read_json(file_path)

# Afficher les premières lignes du DataFrame
print(df.head())


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\d\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\d\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


FileNotFoundError: File train-v2.0.json does not exist

In [ ]:

# Afficher des informations générales sur le DataFrame
print(df.info())


# Explorer les colonnes
print(df.columns)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 442 entries, 0 to 441
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   version  442 non-null    object
 1   data     442 non-null    object
dtypes: object(2)
memory usage: 7.0+ KB
None
Index(['version', 'data'], dtype='object')


In [ ]:

# Explorer la structure JSON si c'est un format imbriqué
if df.columns[0] == 0:
    df = pd.json_normalize(df.to_dict(orient='records'))

# Afficher les premières lignes du DataFrame après normalisation
print(df.head())

# Afficher des informations générales sur le DataFrame après normalisation
print(df.info())

# Explorer les colonnes après normalisation
print(df.columns)


  version                                               data
0    v2.0  {'title': 'Beyoncé', 'paragraphs': [{'qas': [{...
1    v2.0  {'title': 'Frédéric_Chopin', 'paragraphs': [{'...
2    v2.0  {'title': 'Sino-Tibetan_relations_during_the_M...
3    v2.0  {'title': 'IPod', 'paragraphs': [{'qas': [{'qu...
4    v2.0  {'title': 'The_Legend_of_Zelda:_Twilight_Princ...
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 442 entries, 0 to 441
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   version  442 non-null    object
 1   data     442 non-null    object
dtypes: object(2)
memory usage: 7.0+ KB
None
Index(['version', 'data'], dtype='object')


In [ ]:

# Supposons que la colonne texte s'appelle 'text'
# Si la colonne 'text' existe, continuez avec le traitement
if 'text' in df.columns:
    texts = df['text']

    # Tokenization
    tokenized_texts = texts.apply(word_tokenize)
    print(tokenized_texts.head())

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    texts_no_stopwords = tokenized_texts.apply(lambda x: [word for word in x if word.lower() not in stop_words])
    print(texts_no_stopwords.head())

    # Utilisation de spaCy pour le traitement linguistique
    def spacy_process(text):
        doc = nlp(text)
        return [(token.text, token.lemma_, token.pos_, token.is_stop) for token in doc]

    spacy_data = texts.apply(spacy_process)
    print(spacy_data.head())
else:
    print("La colonne 'text' n'existe pas dans le DataFrame.")

La colonne 'text' n'existe pas dans le DataFrame.


In [ ]:
#Revoir la structure de la colonne data
df['data']

0      {'title': 'Beyoncé', 'paragraphs': [{'qas': [{...
1      {'title': 'Frédéric_Chopin', 'paragraphs': [{'...
2      {'title': 'Sino-Tibetan_relations_during_the_M...
3      {'title': 'IPod', 'paragraphs': [{'qas': [{'qu...
4      {'title': 'The_Legend_of_Zelda:_Twilight_Princ...
                             ...                        
437    {'title': 'Infection', 'paragraphs': [{'qas': ...
438    {'title': 'Hunting', 'paragraphs': [{'qas': [{...
439    {'title': 'Kathmandu', 'paragraphs': [{'qas': ...
440    {'title': 'Myocardial_infarction', 'paragraphs...
441    {'title': 'Matter', 'paragraphs': [{'qas': [{'...
Name: data, Length: 442, dtype: object


4) Pétraitement des données
4.1) Tokenizer les contexte et les questions en utilisant hugging face transformer 

In [ ]:
pip install transformers

In [ ]:
# Extraire les question du df en faire une colone

import pandas as pd

# Fonction pour extraire les questions d'une entrée

def extract_questions(entry):
    questions = []
    for paragraph in entry['paragraphs']:
        for qas in paragraph['qas']:
            questions.append(qas['question'])
    return questions

# Appliquer la fonction à chaque ligne de la colonne 'data'

df['questions'] = df['data'].apply(extract_questions)

# Créer un nouveau DataFrame pour exploser les listes de questions tout en respectant les index

questions_expanded = df.explode('questions')

# Afficher le DataFrame résultant
print(questions_expanded)


    version                                               data  \
0      v2.0  {'title': 'Beyoncé', 'paragraphs': [{'qas': [{...   
0      v2.0  {'title': 'Beyoncé', 'paragraphs': [{'qas': [{...   
0      v2.0  {'title': 'Beyoncé', 'paragraphs': [{'qas': [{...   
0      v2.0  {'title': 'Beyoncé', 'paragraphs': [{'qas': [{...   
0      v2.0  {'title': 'Beyoncé', 'paragraphs': [{'qas': [{...   
..      ...                                                ...   
441    v2.0  {'title': 'Matter', 'paragraphs': [{'qas': [{'...   
441    v2.0  {'title': 'Matter', 'paragraphs': [{'qas': [{'...   
441    v2.0  {'title': 'Matter', 'paragraphs': [{'qas': [{'...   
441    v2.0  {'title': 'Matter', 'paragraphs': [{'qas': [{'...   
441    v2.0  {'title': 'Matter', 'paragraphs': [{'qas': [{'...   

                                             questions  
0             When did Beyonce start becoming popular?  
0    What areas did Beyonce compete in when she was...  
0    When did Beyonce leave Destiny'

In [ ]:
from transformers import BertTokenizer

# Charger le tokenizer pour le modèle 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Extraire les contextes et les questions sous forme de listes

contexts = df['data'].apply(lambda x: " ".join([paragraph['context'] for paragraph in x['paragraphs']]))
questions = df['questions'].explode().dropna().tolist()  # Éclater les listes de questions et supprimer les NaN

# Vérifier le contenu
print("Contexts:", contexts.head())
print("Questions:", questions[:5])

# Tokeniser les contextes
context_tokens = tokenizer(contexts.tolist(), padding=True, truncation=True, return_tensors='pt')

# Tokeniser les questions
question_tokens = tokenizer(questions, padding=True, truncation=True, return_tensors='pt')

# Afficher les résultats
print("Context Tokens:")
print(context_tokens)

print("Question Tokens:")
print(question_tokens)


Contexts: 0    Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...
1    Frédéric François Chopin (/ˈʃoʊpæn/; French pr...
2    The exact nature of relations between Tibet an...
3    The iPod is a line of portable media players a...
4    The Legend of Zelda: Twilight Princess (Japane...
Name: data, dtype: object
Questions: ['When did Beyonce start becoming popular?', 'What areas did Beyonce compete in when she was growing up?', "When did Beyonce leave Destiny's Child and become a solo singer?", 'In what city and state did Beyonce  grow up? ', 'In which decade did Beyonce become famous?']
Context Tokens:
{'input_ids': tensor([[  101, 20773, 21025,  ...,  2038,  2180,   102],
        [  101, 15296,  8173,  ...,  5154,  2189,   102],
        [  101,  1996,  6635,  ...,  2029,  5360,   102],
        ...,
        [  101, 28045,  1006,  ...,  1006,  1315,   102],
        [  101,  2026, 24755,  ..., 13923,  1997,   102],
        [  101,  2077,  1996,  ..., 19995,  1012,   102]]), 'token_type_ids

4.2)  Préparer les données pour l'entraînement, en gérant les longueurs des séquences et en créant les étiquettes appropriées

In [ ]:
import pandas as pd
from transformers import BertTokenizerFast
import torch
from torch.utils.data import DataLoader, Dataset

# Charger le tokenizer pour le modèle 'bert-base-uncased'
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

# Fonction pour extraire les contextes, questions, et réponses
def extract_qa_data(entry):
    contexts, questions, answers, start_positions = [], [], [], []
    for paragraph in entry['paragraphs']:
        context = paragraph['context']
        for qas in paragraph['qas']:
            question = qas['question']
            if len(qas['answers']) > 0:
                answer = qas['answers'][0]['text']  # On suppose qu'il y a une seule réponse par question
                answer_start = qas['answers'][0]['answer_start']
                contexts.append(context)
                questions.append(question)
                answers.append(answer)
                start_positions.append(answer_start)
    return contexts, questions, answers, start_positions

# Appliquer la fonction à chaque ligne de la colonne 'data'

contexts, questions, answers, start_positions = [], [], [], []
for i, row in df.iterrows():
    c, q, a, s = extract_qa_data(row['data'])
    contexts.extend(c)
    questions.extend(q)
    answers.extend(a)
    start_positions.extend(s)

# Préparer les données pour le DataLoader
class QADataset(Dataset):
    def __init__(self, contexts, questions, answers, start_positions, tokenizer, max_length=512):
        self.contexts = contexts
        self.questions = questions
        self.answers = answers
        self.start_positions = start_positions
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        context = self.contexts[idx]
        question = self.questions[idx]
        answer = self.answers[idx]
        start_position = self.start_positions[idx]

        # Tokeniser la question et le contexte
        encoding = self.tokenizer(
            question,
            context,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt',
            return_offsets_mapping=True
        )

        # Identifier les positions de début et de fin de la réponse dans les tokens
        offsets = encoding['offset_mapping'][0].numpy()
        start_char = start_position
        end_char = start_position + len(answer)
        start_token, end_token = None, None

        for i, (start, end) in enumerate(offsets):
            if start <= start_char < end:
                start_token = i
            if start < end_char <= end:
                end_token = i
                break

        if start_token is None or end_token is None:
            start_token, end_token = 0, 0

        # Créer les étiquettes
        encoding = {key: val.squeeze() for key, val in encoding.items() if key != 'offset_mapping'}
        encoding['start_positions'] = torch.tensor(start_token, dtype=torch.long)
        encoding['end_positions'] = torch.tensor(end_token, dtype=torch.long)

        return encoding

# Créer un DataLoader
dataset = QADataset(contexts, questions, answers, start_positions, tokenizer)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

# Afficher un batch d'exemples
for batch in dataloader:
    print(batch)
    break


{'input_ids': tensor([[  101,  2054, 11502,  ...,     0,     0,     0],
        [  101,  2054,  2003,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'start_positions': tensor([162,  77]), 'end_positions': tensor([165,  78])}


5)Entraînement du modèle
5.1) Utiliser un modèle pré-entraîné de votre choix pour la tâche de Question Answering

In [ ]:
from transformers import pipeline, BertTokenizerFast, BertForQuestionAnswering

# Charger le tokenizer et le modèle pré-entraîné
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
model = BertForQuestionAnswering.from_pretrained('bert-base-uncased')
qa_pipeline = pipeline('question-answering', model=model, tokenizer=tokenizer)


Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Exemple d'utilisation du pipeline pour la tâche de QA
qa_results = []
for context, question in zip(contexts, questions):
    result = qa_pipeline(question=question, context=context)
    qa_results.append(result)

# Afficher les résultats
for i, result in enumerate(qa_results):
    print(f"Question: {questions[i]}")
    print(f"Context: {contexts[i]}")   
    print(f"Answer: {result['answer']}\n")


RuntimeError: [enforce fail at ..\c10\core\impl\alloc_cpu.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 1179648 bytes.

5.2) Fine-tuner le modèle sur le jeu de données SQuAD 2.0.

In [ ]:
from datasets import load_dataset
from transformers import BertTokenizerFast, BertForQuestionAnswering, Trainer, TrainingArguments

# Charger le jeu de données SQuAD 2.0
dataset = load_dataset("squad_v2")

# Définir le modèle et le tokenizer

model_name = 'bert-base-uncased'
tokenizer = BertTokenizerFast.from_pretrained(model_name)
model = BertForQuestionAnswering.from_pretrained(model_name)

# Définir les arguments d'entraînement
training_args = TrainingArguments(
    per_device_train_batch_size=4,
    num_train_epochs=2,
    logging_dir='./logs',
    logging_steps=100,
    save_steps=1000,
    evaluation_strategy='epoch',
    report_to='wandb',    # Si on utilise Weights & Biases pour le suivi
    save_total_limit=2,
)

# Fonction de prétraitement des données
def preprocess_data(examples):
    return tokenizer(examples['question'], examples['context'], truncation=True, padding='max_length')

# Prétraiter les données
train_dataset = dataset['train'].map(preprocess_data, batched=True)
eval_dataset = dataset['validation'].map(preprocess_data, batched=True)

# Fine-tuning du modèle
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

trainer.train()

# Évaluer le modèle
eval_results = trainer.evaluate()
print(eval_results)


6) Évaluation du modèle